In [1]:
#Now here we mount the drive.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Now here we check the status of gpu.

!nvidia-smi

Thu Aug 15 04:55:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
#Now here we import the kaggle.

!pip install kaggle

In [4]:
#Now here we connect the kaggle with this notebook.

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
#Now here we download the dataset directly in this notebook.

!kaggle datasets download -d paridhijain02122001/cotton-crop-disease-detection

Dataset URL: https://www.kaggle.com/datasets/paridhijain02122001/cotton-crop-disease-detection
License(s): unknown
100% 655M/656M [00:40<00:00, 19.6MB/s]
100% 656M/656M [00:40<00:00, 17.1MB/s]


In [6]:
#Now here we unzip the dataset.

!unzip cotton-crop-disease-detection.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/Bacterial_Blight/aug_1_pil_constract_4.jpg  
  inflating: train/Bacterial_Blight/aug_1_pil_constract_40.jpg  
  inflating: train/Bacterial_Blight/aug_1_pil_constract_5.jpg  
  inflating: train/Bacterial_Blight/aug_1_pil_constract_7.jpg  
  inflating: train/Bacterial_Blight/aug_1_pil_constract_8.jpg  
  inflating: train/Bacterial_Blight/aug_1_pil_constract_9.jpg  
  inflating: train/Bacterial_Blight/aug_1_pil_sharpness_1.jpg  
  inflating: train/Bacterial_Blight/aug_1_pil_sharpness_10.jpg  
  inflating: train/Bacterial_Blight/aug_1_pil_sharpness_11.jpg  
  inflating: train/Bacterial_Blight/aug_1_pil_sharpness_12.jpg  
  inflating: train/Bacterial_Blight/aug_1_pil_sharpness_13.jpg  
  inflating: train/Bacterial_Blight/aug_1_pil_sharpness_14.jpg  
  inflating: train/Bacterial_Blight/aug_1_pil_sharpness_15.jpg  
  inflating: train/Bacterial_Blight/aug_1_pil_sharpness_16.jpg  
  inflating: train/Bacterial_Blight/aug_1_pil

In [7]:
#Now here we import all the important libraries.

#Linear Algebra.
import numpy as np

#Data preprocessing.
import pandas as pd

#tensorflow(acts as backend).
import tensorflow as tf

#models and layers of keras of tensorflow.
from tensorflow.keras import models, layers

#Data Visualization.
import matplotlib.pyplot as plt

#Statistic Visualization.
import seaborn as sns

#Operating system.
import os

#Imagedatagenerator of keras preprocessing.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Image of keras preprocessing.
from tensorflow.keras.preprocessing import image

#Opencv.
import cv2

#Image of pillow.
from PIL import Image

In [8]:
#Now here we adjust the dataset using Imagedatagenerator.

train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)

In [9]:
#Now here we prepare the train and test dataset.

train_ds=train_datagen.flow_from_directory(directory='/content/train',
                                         target_size=(256,256),
                                         batch_size=32,
                                         class_mode='categorical')

test_ds=test_datagen.flow_from_directory(directory='/content/test',
                                         target_size=(256,256),
                                         batch_size=32,
                                         class_mode='categorical')

Found 8640 images belonging to 6 classes.
Found 721 images belonging to 6 classes.


In [10]:
#Now here we check the classes of train dataset.

train_ds.class_indices

{'Aphids': 0,
 'Army_worm': 1,
 'Bacterial_Blight': 2,
 'Healthy': 3,
 'Powdery_Mildew': 4,
 'Target_spot': 5}

In [11]:
#Now here we create the cnn model.

model=models.Sequential([
    layers.Conv2D(32, kernel_size=(3,3), padding='valid', activation='relu', input_shape=(256,256,3)),
    layers.MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'),

    layers.Conv2D(64, kernel_size=(3,3), padding='valid', activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'),

    layers.Conv2D(128, kernel_size=(3,3), padding='valid', activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(6, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
#Now here we compile the model.

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
#Now here we train the model.

history=model.fit(train_ds,
                  epochs=10,
                  batch_size=32,
                  validation_data=test_ds)

Epoch 1/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 0s 508ms/step - accuracy: 0.4700 - loss: 1.3496

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


270/270 ━━━━━━━━━━━━━━━━━━━━ 150s 538ms/step - accuracy: 0.4701 - loss: 1.3493 - val_accuracy: 0.5798 - val_loss: 1.0474
Epoch 2/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 196s 523ms/step - accuracy: 0.5763 - loss: 1.1219 - val_accuracy: 0.7101 - val_loss: 0.7999
Epoch 3/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 201s 520ms/step - accuracy: 0.6760 - loss: 0.9127 - val_accuracy: 0.7282 - val_loss: 0.7463
Epoch 4/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 145s 524ms/step - accuracy: 0.7273 - loss: 0.7647 - val_accuracy: 0.8280 - val_loss: 0.4818
Epoch 5/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 202s 524ms/step - accuracy: 0.7855 - loss: 0.6195 - val_accuracy: 0.8502 - val_loss: 0.4244
Epoch 6/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 199s 513ms/step - accuracy: 0.8250 - loss: 0.5225 - val_accuracy: 0.8821 - val_loss: 0.3352
Epoch 7/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 143s 516ms/step - accuracy: 0.8451 - loss: 0.4516 - val_accuracy: 0.8932 - val_loss: 0.3143
Epoch 8/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 140s 509ms/step - accuracy: 0.8662 - loss: 0.38

In [15]:
#Now here we evaluate the model.

model.evaluate(test_ds)

23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 155ms/step - accuracy: 0.8869 - loss: 0.3407


[0.2965609133243561, 0.9001386761665344]

In [19]:
#Now here we evaluate the model on test image.

img_path=r"/content/test/Army_worm/11.jpg"

#Now here we process the image.

img=image.load_img(img_path, target_size=(256,256))
img_array=image.img_to_array(img)
img_array=np.expand_dims(img_array, axis=0)
img_array=img_array/255.0

#Now here we predict the image.

prediction=model.predict(img_array)
prediction=np.argmax(prediction)

#Now here we print the prediction.
classes=['Aphids','Army_worm', 'Bacterial_Blight','Healthy','Powdery_Mildew','Target_spot']

print(classes[prediction])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Army_worm


In [20]:
#Now here we save the model.

model.save('cotton_disease_detection.h5')